참고 링크
- https://koreanfoodie.me/157
- https://nbviewer.jupyter.org/github/SDRLurker/deep-learning/blob/master/5%E1%84%8C%E1%85%A1%E1%86%BC.ipynb
- 밑바닥부터 시작하는 딥러닝1 책

## 5.6 Affine/Softmax 계층 구현하기
##### 5.6.1 Affine 계층
신경망의 순전파에서는 가중치 신호의 총합을 계산하기 위해 행렬의 내적__(np.dot())__을 사용

In [1]:
import numpy as np

X = np.random.rand(2) # 입력
W = np.random.rand(2, 3) #가중치
B = np.random.rand(3)   #편향

print(X.shape) #(2, )
print(W.shape) #(2, 3)
print(B.shape) #(3, )

Y = np.dot(X, W) + B

(2,)
(2, 3)
(3,)


__위 코드를 계산 그래프를 나타내면 다음과 같다.__
![](img/affine1.png)

> $X$와 $W$의 내적은 대응하는 차원의 원소 수를 일치 시켜야 함<br><br>
__어파인 변환(Affine Transformation) :__ 신경망의 순전파 때 수행하는 행렬의 내적. 기하학 용어<br><br>
이 계산 그래프는 __행렬__이 흐름

<u>__식 5.13 행렬을 사용한 역전파 전개식__</u> : __가장 핵심이라 생각함__
$$
\frac{\partial L}{\partial X} = \frac{\partial L}{\partial Y}W^T
$$
$$
\frac{\partial L}{\partial W} = X^T\frac{\partial L}{\partial Y}
$$

$W^T$는 행렬$W$의 전치행렬을 의미한다.

##### 5.6.2 배치용 aFFINE 계층

지금까지의 Affine 계층은 입력데이터로 $X$ 하나만을 고려한 것이었다. <br>
이번 절에서는 __데이터 N개를 묶어 순전파__ 하는 경우, 즉 배치용(데이터 묶음) Affine 계층을 생각해 보자.
![](img/affine3.png)

<u>__기존과 다른 부분은 입력인 $X$의 형상이 (N,2)가 된 것 뿐이다.__</u> <br>
그 뒤로는 지금까지와 같이 계산 그래프의 순서를 따라 순순히 행렬 계산을 하게 된다.


예를 들어 N=2(데이터가 2개)로 한 경우, 편향은 그 두 데이터 각각에 더해집니다.

In [6]:
X_dot_W = np.array([[0, 0, 0], [10, 10, 10]])
B = np.array([1, 2, 3])

X_dot_W

array([[ 0,  0,  0],
       [10, 10, 10]])

In [7]:
X_dot_W + B

array([[ 1,  2,  3],
       [11, 12, 13]])

순전파의 편향 덧셈은 각각의 데이터(1번째 데이터, 2번째 데이터)에 더해짐<br><br>

역전파 때는 각 데이터의 역전파 값이 편향의 원소에 모여야 함

In [8]:
dY = np.array([[1, 2, 3], [4, 5, 6]])
dY

array([[1, 2, 3],
       [4, 5, 6]])

In [9]:
dB = np.sum(dY, axis=0)
dB

array([5, 7, 9])

##### Affine 구현

common/layer.py 파일의 Affine 구현은 입력 데이터가 텐서(4차원 데이터)인 경우도 고려. 다음 구현과 약간 차이가 있다. 

![](img/affine4.png)
__Affine 코드 이해를 위한 그림__

In [10]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis = 0)
        
        return dx

##### Softmax-with-Loss 계층

__1) 신경망은 추론과 학습으로 구성__
- 1. 추론 : 가장 좋은 결과 찾기(softmax함수와 같은 활성하ㅗ 함수를 사용하지 않고 점수 그대로 사용)
- 2. 학습 : 변수를 수정해서 추론에서 높은 값을 확률 1에 가깝게 만들고, 낮은 값은 확률 0이 되게 함
    - 이 중, 학습에 필요한 것이 Softmax함수이다.

__2) Softmax 함수는 입력값을 확률 형태로 normalize하여 결과값 출력 (출력값을 이용하여 모델 업데이트)__


- 정답지(Label)이 있어야 학습이 되어 차이값을 앞 W,B값에 반영
    - ex) 들어온 추론에서는 3이 가장 높은 값이었고 확률이 0.991이었다면, W, B를 조절해서 확률을 1에 가깝게 함
    - => Softmax 결과를 손실함수로 오차 계산 (by Cross Entropy Error)
    - 활성화함수는 Mean Squared Error를 통해 오차 계산
    
__3) Softmax__
- __결과에 대한 값의 합이 1이 되도록 정규화시켜주는 역할도 함__

In [12]:
def softmax(a) :
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

a = np.array([0.3, 2.9, 4.0])

print(softmax(a))              # softmax 결과값 출력
print(sum(softmax(a)))         # 결과값은 합은 1이 됨 

[0.01821127 0.24519181 0.73659691]
1.0


In [13]:
# https://github.com/WegraLee/deep-learning-from-scratch/blob/master/common/functions.py 소스 참고
# 3.5.2 소프트맥스 함수 구현시 주의점 참고
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# 4.2.2. 교차 엔트로피 오차 참고
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 훈련 데이터가 원-핫 벡터라면 정답 레이블의 인덱스로 반환
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_size

class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 손실
        self.y = None    # softmax의 출력
        self.t = None    # 정답 레이블(원-핫 벡터)
    
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        
        return dx

__※ 주의점 :  역전파 때는 전파하는 값을 배치의 수(batch_size)로 나눠, 데이터 1개당 오차를 앞 계층으로 전파함__